In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
! pip install -q onnxruntime onnx==1.14.0 onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *
from tqdm.notebook import tqdm
import numpy as np
import json

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/New folder

/content/drive/MyDrive/New folder


In [7]:
df = pd.read_csv("final_details.csv")
df.head()

,title,url,description,types
0,Cucumber Sandwiches with Fresh Goat Cheese,https://eatsmarter.com/recipes/cucumber-sandwiches-with-fresh-goat-cheese,"Due to the high water content, the vegetables are very helpful for weight loss. 100 grams of cucumber provide just 12 calories. Cucumbers also contain potassium, iron, zinc, fluorine, and some vitamin C. With the goat's cheese comes satiating protein between the slices.","['Cheese', 'Vegetable', 'low-carb']"
1,Smoked Trout Tartare with Avocado Mango Salsa,https://eatsmarter.com/recipes/smoked-trout-tartare-with-avocado-mango-salsa,"The sulfur substances contained in chives are responsible for their slight pungency. They provide a slight antibacterial effect and have a positive effect on digestion. If things run smoothly in the intestines, our concentration is also ensured. This is also supported by the omega-3 fatty acids from the smoked salmon trout.","['Finger Food', 'Low-cholesterol', 'Fitness', 'Healthy Eating', 'Low-calorie', 'Low-calorie Meat Dish', '250-400 Calorie', 'low-carb', 'non-alcoholic', 'egg-free', 'Pescetarian', 'Special', '25-Minute', 'Buffet', 'Exotic', 'Cooking on vacation', 'festively', 'Meal for Four', 'Garden Party', 'Guest', 'Party', 'Picnic', 'Cooking', 'Fish', 'Vegetable', 'leafy green vegetable', 'Onion', 'Herb', 'Dairy', 'Fruit', 'Brunch', 'Snack', 'Party Snack', 'Party Snack', 'Tapas', 'Appetizer']"
2,Basic Tofu Recipe,https://eatsmarter.com/recipes/basic-tofu-recipe,"Do you have a sensitive stomach? Then tofu is perfect for you, because it's easy on the stomach and doesn't put a strain on digestion. Great veggie option: With around 15 percent vegetable protein, soy cheese gives us about as much as most types of meat offer.","['Tofu', 'nutritional', 'Low-cholesterol', 'High-protein', 'Healthy Eating', 'Gluten-free', 'Gluten-free Vegan Recipe', 'lactose-free', 'low-carb', 'Mineral-rich', 'non-alcoholic', 'egg-free', 'meat-free', 'Milk-free', 'Vegan', 'Vegetarian', 'regional', 'Asian', 'Chinese', 'Japanese', 'Special', 'Cooking on vacation', 'Cooking', 'Soy']"
3,Grilled Watermelon with Herbed Ricotta,https://eatsmarter.com/recipes/grilled-watermelon-with-herbed-ricotta,"Despite the fruity-sweet taste of Watermelons there is relatively little sugar in it. On top, the fruit provides plenty of liquid for a refreshing feast on hot summer days. Essential oils from Cress help to get disease-causing bacteria under control. For example, cold symptoms can subside more quickly.","['vegetarian grilling', 'nutritional', 'Clean Eating', 'Healthy Eating', 'Gluten-free', 'Gluten-free Snack', 'Gluten-free Vegetarian Recipe', 'Gluten-free Dinner', 'Gluten-free Lunch', '100-250 Calorie', 'low-carb', 'low-carb dinner', 'low-carb lunch', 'low-carb vegetarian', 'low-carb salad', 'low-carb snack', 'Vegetarian', 'Vegetarian Lunch', 'Vegetarian Dinner', 'Vegetarian Salad', 'Vegetarian Appetizers', 'Vegetarian Snack', 'regional', 'European', 'seasonal', 'Summer', 'grilled', 'Barbecue', 'grilling side dishes', 'cookout', 'grilling', 'Grilled Fruit', 'Course', 'Lunch', 'Dinner', 'M..."
4,Egg Bites with Smoked Salmon and Broccoli,https://eatsmarter.com/recipes/egg-bites-with-smoked-salmon-and-broccoli,"Eggs contain a lot of biotin. This B vitamin promotes hair growth and a healthy nail bed. In addition, the substance is needed for all metabolic processes in the organism.","['muffin', 'Baking', 'pastry', 'nutritional', 'Gluten-free', 'Gluten-free Snack', 'Gluten-free Fish Recipe', 'low-carb', 'regional', 'North America', 'seasonal', 'Summer', 'Autumn', 'Course', 'Brunch', 'Snack', 'Snack', 'Nibble', 'Finger Food', 'Appetizer', 'Party Snack', 'Party Snack', 'Menu', 'Appetizer']"


In [8]:
df=df.drop_duplicates(subset=['url'])
df = df[df.types != '[]']

df = df.dropna().reset_index(drop=True)
df.shape

(18396, 4)

In [9]:
import ast

for i in range(len(df.types)):
  try:
    list_1 = ast.literal_eval(df.types[i])
    if isinstance(list_1, list):
      df.types[i] = list_1
  except (ValueError, SyntaxError):
    pass

In [10]:
types_list = df.types.to_list()

In [11]:
mapping = {
    'Meal for Four': 'Family Meal',
    'Onion': 'Vegetable',
    'Poultry': 'Chicken',
    'Course': 'Main Course',
    'Osteoarthritis': 'Reduce Osteoarthritis',
    'Stress': 'Reduce Stress',
    'Iron Deficiency': 'Reduce Iron Deficiency',
    'Cooking on vacation': 'vacation',
    'saltwater\xa0Fish': 'saltwater fish',
    'Root Vegetable': 'Vegetable',
    'Pregnancy': 'Good during Pregnancy',
    'Fertility': 'Improves Fertility'
}

for sublist in types_list:
    for i in range(len(sublist)):
        sublist[i] = mapping.get(sublist[i], sublist[i])

types_list[1]

['Finger Food',
 'Low-cholesterol',
 'Fitness',
 'Healthy Eating',
 'Low-calorie',
 'Low-calorie Meat Dish',
 '250-400 Calorie',
 'low-carb',
 'non-alcoholic',
 'egg-free',
 'Pescetarian',
 'Special',
 '25-Minute',
 'Buffet',
 'Exotic',
 'vacation',
 'festively',
 'Family Meal',
 'Garden Party',
 'Guest',
 'Party',
 'Picnic',
 'Cooking',
 'Fish',
 'Vegetable',
 'leafy green vegetable',
 'Vegetable',
 'Herb',
 'Dairy',
 'Fruit',
 'Brunch',
 'Snack',
 'Party Snack',
 'Party Snack',
 'Tapas',
 'Appetizer']

In [12]:
for sublist in types_list:
    new_sublist = []
    for item in sublist:
        if item not in ['Menu', 'Guest', 'Easy', 'Lactation', 'Quick','Cooking']:
            new_sublist.append(item)
    sublist[:] = new_sublist
types_list[1]

['Finger Food',
 'Low-cholesterol',
 'Fitness',
 'Healthy Eating',
 'Low-calorie',
 'Low-calorie Meat Dish',
 '250-400 Calorie',
 'low-carb',
 'non-alcoholic',
 'egg-free',
 'Pescetarian',
 'Special',
 '25-Minute',
 'Buffet',
 'Exotic',
 'vacation',
 'festively',
 'Family Meal',
 'Garden Party',
 'Party',
 'Picnic',
 'Fish',
 'Vegetable',
 'leafy green vegetable',
 'Vegetable',
 'Herb',
 'Dairy',
 'Fruit',
 'Brunch',
 'Snack',
 'Party Snack',
 'Party Snack',
 'Tapas',
 'Appetizer']

In [13]:
for sublist in types_list:
    sublist[:] = list(set(sublist))
types_list[1]

['Party Snack',
 'Buffet',
 'Fitness',
 'Vegetable',
 'Fish',
 'Snack',
 'Low-calorie Meat Dish',
 'Pescetarian',
 '25-Minute',
 'Dairy',
 'Special',
 'Low-calorie',
 'Tapas',
 'Picnic',
 'Finger Food',
 'low-carb',
 'non-alcoholic',
 'Herb',
 'Low-cholesterol',
 'egg-free',
 'Party',
 'Brunch',
 'Garden Party',
 'Appetizer',
 'festively',
 'Exotic',
 'leafy green vegetable',
 '250-400 Calorie',
 'vacation',
 'Healthy Eating',
 'Fruit',
 'Family Meal']

In [14]:
df.types=types_list
df

,title,url,description,types
0,Cucumber Sandwiches with Fresh Goat Cheese,https://eatsmarter.com/recipes/cucumber-sandwiches-with-fresh-goat-cheese,"Due to the high water content, the vegetables are very helpful for weight loss. 100 grams of cucumber provide just 12 calories. Cucumbers also contain potassium, iron, zinc, fluorine, and some vitamin C. With the goat's cheese comes satiating protein between the slices.","[Vegetable, low-carb, Cheese]"
1,Smoked Trout Tartare with Avocado Mango Salsa,https://eatsmarter.com/recipes/smoked-trout-tartare-with-avocado-mango-salsa,"The sulfur substances contained in chives are responsible for their slight pungency. They provide a slight antibacterial effect and have a positive effect on digestion. If things run smoothly in the intestines, our concentration is also ensured. This is also supported by the omega-3 fatty acids from the smoked salmon trout.","[Party Snack, Buffet, Fitness, Vegetable, Fish, Snack, Low-calorie Meat Dish, Pescetarian, 25-Minute, Dairy, Special, Low-calorie, Tapas, Picnic, Finger Food, low-carb, non-alcoholic, Herb, Low-cholesterol, egg-free, Party, Brunch, Garden Party, Appetizer, festively, Exotic, leafy green vegetable, 250-400 Calorie, vacation, Healthy Eating, Fruit, Family Meal]"
2,Basic Tofu Recipe,https://eatsmarter.com/recipes/basic-tofu-recipe,"Do you have a sensitive stomach? Then tofu is perfect for you, because it's easy on the stomach and doesn't put a strain on digestion. Great veggie option: With around 15 percent vegetable protein, soy cheese gives us about as much as most types of meat offer.","[Tofu, Japanese, Asian, High-protein, lactose-free, Gluten-free, Gluten-free Vegan Recipe, meat-free, Special, Chinese, Milk-free, low-carb, non-alcoholic, Low-cholesterol, Vegan, Soy, egg-free, Vegetarian, Mineral-rich, regional, vacation, Healthy Eating, nutritional]"
3,Grilled Watermelon with Herbed Ricotta,https://eatsmarter.com/recipes/grilled-watermelon-with-herbed-ricotta,"Despite the fruity-sweet taste of Watermelons there is relatively little sugar in it. On top, the fruit provides plenty of liquid for a refreshing feast on hot summer days. Essential oils from Cress help to get disease-causing bacteria under control. For example, cold symptoms can subside more quickly.","[Vegetarian Salad, seasonal, low-carb dinner, Vegetarian Appetizers, Gluten-free, Gluten-free Snack, Lunch, low-carb lunch, low-carb snack, Vegetarian Snack, Clean Eating, Gluten-free Dinner, grilled, Dinner, Gluten-free Lunch, Summer, low-carb, grilling, Vegetarian Lunch, low-carb vegetarian, grilling side dishes, low-carb salad, Vegetarian, Vegetarian Dinner, Main Course, Appetizer, Gluten-free Vegetarian Recipe, Grilled Fruit, European, vegetarian grilling, regional, Healthy Eating, 100-250 Calorie, nutritional, Barbecue, cookout]"
4,Egg Bites with Smoked Salmon and Broccoli,https://eatsmarter.com/recipes/egg-bites-with-smoked-salmon-and-broccoli,"Eggs contain a lot of biotin. This B vitamin promotes hair growth and a healthy nail bed. In addition, the substance is needed for all metabolic processes in the organism.","[Party Snack, seasonal, Gluten-free Fish Recipe, Snack, Gluten-free, pastry, Gluten-free Snack, Autumn, Nibble, Finger Food, Summer, Baking, low-carb, North America, Brunch, Main Course, Appetizer, muffin, regional, nutritional]"
...,...,...,...,...
18391,Sausage and Ham Croissants,https://eatsmarter.com/recipes/sausage-and-ham-croissants,Roll the dough on a lightly floured surface to 4-5 mm (approximately 1/8 inch) thick. Cut into 3 cm (approximately 1 1/4 inch) wide and 20 cm (approximately 8 inch) long strips.,"[Quark-oil Dough, Eggs, Milk, Spices, Breakfast, Grain, 250-400 Calorie, Dairy, Pork]"
18392,Shakshuka with Eggplant,https://eatsmarter.com/recipes/shakshuka-with-eggplant,This delicious vegetable and egg shakshuka dish is packed with a slew of vitamins and minerals thanks to the vegetable mixture as well as powerful protein from the eggs. Shakshukas ar

In [15]:
types_list = df.types.to_list()
type_count = {}
for type1 in types_list:
  if isinstance(type1, list):
    type_list = type1
  else:
    type_list = eval(type1)
  for type2 in type_list:
    if type2 in type_count.keys():
      type_count[type2] += 1
    else:
      type_count[type2] = 1
print(f"Number of types: {len(type_count)}")
print(type_count)

threshold = int(len(df) * 0.05)
rare_types = [key for key, value in type_count.items() if value < threshold]
len(rare_types)

Number of types: 1095
{'Vegetable': 7173, 'low-carb': 5870, 'Cheese': 973, 'Party Snack': 166, 'Buffet': 245, 'Fitness': 171, 'Fish': 1938, 'Snack': 1307, 'Low-calorie Meat Dish': 8, 'Pescetarian': 266, '25-Minute': 179, 'Dairy': 764, 'Special': 200, 'Low-calorie': 519, 'Tapas': 84, 'Picnic': 378, 'Finger Food': 362, 'non-alcoholic': 2169, 'Herb': 2768, 'Low-cholesterol': 846, 'egg-free': 875, 'Party': 674, 'Brunch': 910, 'Garden Party': 472, 'Appetizer': 1231, 'festively': 346, 'Exotic': 308, 'leafy green vegetable': 841, '250-400 Calorie': 876, 'vacation': 1759, 'Healthy Eating': 1303, 'Fruit': 1950, 'Family Meal': 1438, 'Tofu': 117, 'Japanese': 104, 'Asian': 1317, 'High-protein': 1639, 'lactose-free': 1157, 'Gluten-free': 981, 'Gluten-free Vegan Recipe': 208, 'meat-free': 733, 'Chinese': 179, 'Milk-free': 465, 'Vegan': 947, 'Soy': 181, 'Vegetarian': 2404, 'Mineral-rich': 1064, 'regional': 596, 'nutritional': 1005, 'Vegetarian Salad': 356, 'seasonal': 698, 'low-carb dinner': 324, 'Ve

1040

In [16]:
types_list = df.types.to_list()
revised_type_list = []
indices_to_drop = []

for idx, types in enumerate(types_list):
  if isinstance(types, list):
    type_list = types
  else:
    type_list = eval(types)
  revised_types = []

  for type1 in type_list:
    if type1 not in rare_types:
      revised_types.append(type1)

  if len(revised_types) == 0:
    indices_to_drop.append(idx)
  else:
    revised_type_list.append(revised_types)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_types'] = revised_type_list
df.shape

(18375, 5)

In [19]:
revised_types_list = df.revised_types.to_list()
revised_type_count = {}
for types in revised_types_list:
  type_list = types
  for type1 in type_list:
    if type1 in revised_type_count.keys():
      revised_type_count[type1] += 1
    else:
      revised_type_count[type1] = 1
print(f"Number of types: {len(revised_type_count)}")

Number of types: 55


In [20]:
revised_type_count.keys()

dict_keys(['Vegetable', 'low-carb', 'Cheese', 'Fish', 'Snack', 'non-alcoholic', 'Herb', 'Appetizer', 'vacation', 'Healthy Eating', 'Fruit', 'Family Meal', 'Asian', 'High-protein', 'lactose-free', 'Gluten-free', 'Vegan', 'Vegetarian', 'Mineral-rich', 'nutritional', 'Lunch', 'Dinner', 'Summer', 'Main Course', 'European', 'Baking', 'Dessert', 'high-fiber', 'Spring', 'Improves Fertility', 'Reduce Iron Deficiency', 'soup', 'Reduce Stress', 'Low-sugar', 'Good during Pregnancy', 'Reduce Osteoarthritis', 'Diabetic', 'diet', 'Vitamin-rich', 'Pasta', 'high cholesterol', 'saltwater fish', 'Vegetable Salad', 'Meat', 'salad', 'Eggs', 'Spices', 'Side Dish', 'Italian', 'fruit-vegetable', 'baked dish', 'Chicken', 'Roast', 'Dough', 'Rice'])

In [21]:
encode_types = { key: idx for idx, (key, value) in enumerate(revised_type_count.items())}
with open("types_encoded.json", "w") as fp:
  json.dump(encode_types, fp)

categorical_types_list = []
revised_types_list = df.revised_types.to_list()

for revised_types in revised_types_list:
  categorical_list = [0] * len(encode_types)
  for genre in revised_types:
    type_index = encode_types[genre]
    categorical_list[type_index] = 1
  categorical_types_list.append(categorical_list)

df['genre_cat_list'] = categorical_types_list
df.shape

(18375, 6)

In [22]:
labels = list(encode_types.keys())
len(labels), labels[:5]

(55, ['Vegetable', 'low-carb', 'Cheese', 'Fish', 'Snack'])

In [24]:
splitter = RandomSplitter(valid_pct=0.15, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(15619, 2756)

In [25]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,url,description,types,revised_types,genre_cat_list
16542,Bacon Wrapped Potatoes with Asparagus and Herb Butter,https://eatsmarter.com/recipes/bacon-wrapped-potatoes-with-asparagus-and-herb-butter,Melt the butter in a pan and add the chopped herbs. Season with salt and pepper.,"[Eggs, Buffet, Vegetable, baked dish, Pancake, salad, Sprout]","[Eggs, Vegetable, baked dish, salad]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
15886,Egg Salad,https://eatsmarter.com/recipes/egg-salad-0,Rinse and peel the tomatoes. Rinse and trim the scallion and cut into rings.,"[salad, Eggs]","[salad, Eggs]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
16239,"Cucumber, Egg and Smoked Salmon Salad",https://eatsmarter.com/recipes/cucumber-egg-and-smoked-salmon-salad,"Arrange the cucumber slices, eggs and smoked salmon on plates and drizzle with the dressing. Season with salt and pepper and garnish with cress.","[Home Cooking, Vegetable, Fish, Vegetable Salad, cucumber salad, salad]","[Vegetable, Fish, Vegetable Salad, salad]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
18313,Blueberry Jam,https://eatsmarter.com/recipes/blueberry-jam-5,"Rinse, dry, and tie the lemon thyme together. Remove 1250 ml (approximately 1 1/4 quarts) of the juice. Squeeze the lime and add the juice. Stir the jam sugar into the juice mixture and add the lemon-thyme. Bring to a boil, and cook for 4 minutes, stirring constantly. Remove from the heat and remove the thyme.","[Jam, Sweet, Pressure cooking, Herb, Spread, Blackberry Jam, Rustic, marmalade, Breakfast, Sweet Breakfast, Sunday cooking, Fruit, Brunch, Garden Party, Blueberry Jam, Country Style]","[Herb, Fruit]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5687,Halibut with Vegetables,https://eatsmarter.com/recipes/halibut-with-vegetables-0,"Rinse fish, cut into four equal pieces, sprinkle with lemon juice, season with salt and pepper and coat with remaining oil. Arrange on top of vegetables and roast in preheated oven at 200°C (approximately 400°F) for about 8 minutes.","[fruit-vegetable, lactose-free, Fish, Gluten-free, saltwater fish, low-carb]","[fruit-vegetable, lactose-free, Fish, Gluten-free, saltwater fish, low-carb]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"


In [28]:
model_path = "/content/drive/MyDrive/New folder/models/recipe-classifier-stage-8.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("random placeholder")

[{'labels': ['Fruit', 'European', 'Summer', 'Easy'],
  'scores': [1.0, 1.0, 1.0, 1.0],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Cheese', 'Vegetable', 'low-carb', 'Healthy Eating', 'non-alcoholic', 'Guest', 'Cooking', 'Fish', 'Onion', 'Herb', 'Fruit', 'Snack', 'Appetizer', 'nutritional', 'High-protein', 'Gluten-free', 'lactose-free', 'Mineral-rich', 'Vegan', 'Vegetarian', 'Asian', 'European', 'Summer', 'Course', 'Lunch', 'Dinner', 'Menu', 'Baking', 'high-fiber', 'Spring', 'Dessert', 'diet', 'Vitamin-rich', 'Low-sugar', 'Diabetic', 'Lactation', 'Pregnancy', 'soup', 'Quick', 'Pasta', 'high cholesterol', 'Main

In [29]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

['Fruit', 'Summer', 'European']

In [32]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['genre_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [30]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_types = [0] * len(encode_types)
  for label in labels:
    pred_types[encode_types[label]] = 1
  preds.append(pred_types)

preds[0][:20]

  0%|          | 0/2756 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [33]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.0752040336708969
F1 Score (Macro) = 0.0103822604090844


In [34]:
model_path = "/content/drive/MyDrive/New folder/models/recipe-classifier-stage-8.pkl"
learner_inf = load_learner(model_path)

In [35]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    '/content/drive/MyDrive/New folder/models/recipe2-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
!pip install -q onnxruntime.quantization

ERROR: Could not find a version that satisfies the requirement onnxruntime.quantization (from versions: none)
ERROR: No matching distribution found for onnxruntime.quantization


In [42]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = '/content/drive/MyDrive/New folder/models/recipe2-classifier.onnx'
quantized_onnx_model_path = '/content/drive/MyDrive/New folder/models/recipe2-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

In [37]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_types.keys())

inf_session = rt.InferenceSession('/content/drive/MyDrive/New folder/models/recipe2-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [38]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_types)
  for label in labels:
    pred_genres[encode_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2756 [00:00<?, ?it/s]

In [40]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.09921804896808102
F1 Score (Macro) = 0.013169079445680227


In [43]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_types.keys())

inf_session = rt.InferenceSession('/content/drive/MyDrive/New folder/models/recipe2-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [44]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_types)
  for label in labels:
    pred_genres[encode_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2756 [00:00<?, ?it/s]

In [45]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.09921804896808102
F1 Score (Macro) = 0.013169079445680227
